In [ ]:
### 다중선형회귀 연습문제
## 관측된 Solar.R, Wind, Temp 대한 Ozone량에 대한 데이터를 
### 이용하여 머신 러닝 진행하고
## Solar.R : 170 , Wind : 7.4, Temp:67.0 에 대한
### Ozone 수치를 예측하세요~~!! => ??
# df
new_df = pd.read_csv("./data/ozone.csv")
new_df

new_df = new_df[["Ozone", "Solar.R", "Wind", "Temp"]]
new_df

new2_df = new_df.dropna(how="any", inplace=False )
new2_df

new_q1,new_q3 = np.percentile(df["Ozone"],[25,75])

new_iqr = new_q3 - new_q1
new_upper = new_q3 + new_iqr * 1.5 # 상위 이상치 처리

new_mask = new2_df["Ozone"] > new_upper # boolean mask 를 생성 

new2_df = new2_df.loc[~new_mask]

print(new2_df)

#x1_data = new2_df[["Solar.R", "Wind", "Temp"]]
#y1_data = new2_df["Ozone"]
                                                            
# 다중 선형회귀 구조는 이렇게 수식을 세워야 됨
# H = X * W + b (표현식)

# # 5. cost function
# cost1 = tf.reduce_mean(tf.square(H-Y1))
# # 6. train
# optimizer1 = tf.train.GradientDescentOptimizer(learning_rate=0.01)
# train1 = optimizer1.minimize(cost1)

# # 7. session, 초기화
# sess = tf.Session()

# sess.run(tf.global_variables_initializer())

# # for step in range(3000):
# #     _, new_cost_val = sess.run([train1,cost1],
# #                           feed_dict={ x : x1_data,
# #                                       y : y1_data})

In [ ]:
# 표준화
new2_df["Temp_Norm"] = (new2_df["Temp"] - new2_df["Temp"].min()) / (new2_df["Temp"].max() - new2_df["Temp"].min())

new2_df["Ozone_Norm"] = (new2_df["Ozone"] - new2_df["Ozone"].min()) / (new2_df["Ozone"].max() - new2_df["Ozone"].min())
new2_df["Solar_Norm"] = (new2_df["Solar.R"] - new2_df["Solar.R"].min()) / (new2_df["Solar.R"].max() - new2_df["Solar.R"].min())
new2_df["Wind_Norm"] = (new2_df["Wind"] - new2_df["Wind"].min()) / (new2_df["Wind"].max() - new2_df["Wind"].min())

print(new2_df)

In [ ]:
# 1. 학습데이터
x1_data = new2_df[["Temp_Norm", "Solar_Norm", "Wind_Norm"]].values
y1_data = new2_df[["Ozone_Norm"]].values

#print(x1_data)
#print(y1_data)

# 2. placeholder
X1 = tf.placeholder(shape=[None,3], dtype=tf.float32)
Y1 = tf.placeholder(shape=[None,1], dtype=tf.float32)

# 3. weight & bias
W = tf.Variable(tf.random_normal([3,1]), name="weight")
b = tf.Variable(tf.random_normal([1]), name="bias")

# 4. 다중 선형 회귀 수식 Hypothesis
H = tf.matmul(X1,W) + b

# 5. cost function
cost1 = tf.reduce_mean(tf.square(H-Y1))

# 6. train
optimizer1 = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train1 = optimizer1.minimize(cost1)

# 7. session, 초기화
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for step in range(30000):
    _, new_cost_val = sess.run([train1,cost1],
                          feed_dict={ X1 : x1_data,
                                      Y1 : y1_data})
    
    if step % 300 == 0:
        print("cost val : {}".format(new_cost_val))

# 최적의 W와 b를 구했어요! -> 가설 완성
# prediction
# 이제 Solar.R: 170 Wind:7.4 Temp:67.0
#print(sess.run(H, feed_dict={x : 80}))

# 이렇게 처리하면 문제가 없어요
# 80이란 온도를 0~1사이의 값으로 스케일
tmp1 = (170 - new2_df["Solar.R"].min()) / (new2_df["Solar.R"].max() - new2_df["Solar.R"].min())
tmp2 = (67 - new2_df["Temp"].min()) / (new2_df["Temp"].max() - new2_df["Temp"].min())
tmp3 = (7.4 - new2_df["Wind"].min()) / (new2_df["Wind"].max() - new2_df["Wind"].min())
# print(tmp1)
# print(tmp2)
# print(tmp3)

normTmp = [[tmp2, tmp1, tmp3]]
print(normTmp)

new_result_scale = sess.run(H, feed_dict= { X1 : normTmp })

new_result = new_result_scale * (new2_df["Ozone"].max() - new2_df["Ozone"].min()) + new2_df["Ozone"].min()

print(new_result)

